In [10]:
import tensorflow as tf
import tensorflow
import tqdm.notebook as note
import numpy as np
import scipy
import re

In [11]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [12]:
with open("drive/MyDrive/bible.txt",'r') as f:
    text = f.read()

text = re.sub(r'\s(?=\s)|[^\sa-zA-Z0-9]+','',text.lower())
text = text.split()

In [13]:
words, _, counts = tf.unique_with_counts(text)
words, counts = np.asarray(words.numpy(), dtype='U'), counts.numpy()
words = words[counts >= sorted(counts)[-10000]]

In [14]:
def build_vocab(words):
    return {a: i for i, a in enumerate(words)}

def build_index2token(vocab):
    return {value:key for key,value in vocab.items()}

In [15]:
d = []
for i in range(len(text)):
    for j in [-2, -1, 1, 2]:
        if i + j >= 0 and i + j < len(text):
            d.append((text[i], text[i + j]))
d = np.asarray(d)

ds = tf.data.Dataset.from_tensor_slices((d[:,0], d[:,1]))

ds = ds.shuffle(1000)
ds = ds.batch(128)
ds = ds.prefetch(20)

In [16]:
class SkipGramModel(tf.keras.layers.Layer):
    def __init__(self, embedding_size, vocabulary_size):
        super(SkipGramModel, self).__init__()
        self.embedding_size = embedding_size
        self.vocabulary_size = vocabulary_size

        self.nce_weights = tf.Variable(tf.random.normal([self.vocabulary_size, self.embedding_size]))
        self.nce_biases = tf.Variable(tf.zeros([self.vocabulary_size]))

        self.embedding = tf.Variable(tf.random.uniform([self.vocabulary_size, self.embedding_size]))

    def call(self, words, labels):
      number_of_negative_samples = 64
      words_embedded = tf.nn.embedding_lookup(self.embedding, words)

      sampled_values=tf.random.fixed_unigram_candidate_sampler(complete_data_to_be_sampled,self.vocabulary_size,number_of_negative_samples,False,range_max=self.vocabulary_size,unigrams=new_counts)

      return tf.reduce_mean(tf.nn.nce_loss(weights=self.nce_weights,biases=self.nce_biases, labels=labels,  inputs=words_embedded,  num_sampled=number_of_negative_samples, num_classes=self.vocabulary_size,sampled_values=sampled_values))

In [17]:
def print_closest_word(Model,words_of_interest):
    ar = Model.embedding.numpy()
    for word in words_of_interest:

        i = ds[word]

        closest = np.argmin([scipy.spatial.distance.cosine(ar[i],ar[q]) if q!=i else np.inf for q in range(len(ar)) ])



tf.keras.backend.clear_session()

words_of_interesting = ["holy", "father", "wine", "poison", "love", "strong", "day"]

learning_rate = 0.001

num_epochs = 5
hidden_size = 64

Model = SkipGramModel(hidden_size, len(ds))

optimizer = tf.optimizers.Adam(learning_rate)    

train_losses = []

# training of our model for num_epochs
for epoch in range(num_epochs):
    print(f'Epoch {str(epoch)}')

    epoch_loss_agg = np.empty(0)

    for input, target in note.tqdm(ds, position=0,leave=True):
      
      with tf.GradientTape() as tape:
        loss = Model(input,target)

      gradients = tape.gradient(loss, Model.trainable_variables)
      optimizer.apply_gradients(zip(gradients, Model.trainable_variables))
      epoch_loss_agg = np.append(epoch_loss_agg, loss)
                   
    train_losses = np.append(train_losses, tf.reduce_mean(epoch_loss_agg))            
                   
    print(f"Finished epoch {epoch+1} with loss: {train_losses[-1]}, closest word is:")
    print_closest_word(Model,words_of_interesting)

Epoch 0


  0%|          | 0/25660 [00:00<?, ?it/s]

ValueError: ignored